In [ ]:
!pip install torch torchvision matplotlib


In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision
import matplotlib
from torchvision import datasets, transforms
from torch.utils.data import DataLoader
import matplotlib.pyplot as plt

print("Torch version:", torch.__version__)
print("Torchvision version:", torchvision.__version__)
print("Matplotlib version:", matplotlib.__version__)
print("CUDA available:", torch.cuda.is_available())


In [ ]:
transform = transforms.Compose([
    transforms.ToTensor()
])

train_dataset = datasets.MNIST(root='./data', train=True, download=True, transform=transform)
test_dataset = datasets.MNIST(root='./data', train=False, download=True, transform=transform)

train_loader = DataLoader(train_dataset, batch_size=128, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=128, shuffle=False)



In [ ]:
class VAE(nn.Module):
    def __init__(self, latent_dim=20):
        super().__init__()

        self.encoder = nn.Sequential(
            nn.Linear(28*28, 400),
            nn.ReLU()
        )

        self.mu = nn.Linear(400, latent_dim)
        self.logvar = nn.Linear(400, latent_dim)

        self.decoder = nn.Sequential(
            nn.Linear(latent_dim, 400),
            nn.ReLU(),
            nn.Linear(400, 28*28),
            nn.Sigmoid()
        )

    def reparameterize(self, mu, logvar):
        std = torch.exp(0.5 * logvar)
        eps = torch.randn_like(std)
        return mu + eps * std

    def forward(self, x):
        x = x.view(-1, 28*28)
        h = self.encoder(x)
        mu = self.mu(h)
        logvar = self.logvar(h)
        z = self.reparameterize(mu, logvar)
        recon = self.decoder(z)
        return recon, mu, logvar


In [ ]:
def vae_loss(recon_x, x, mu, logvar):
    recon_loss = nn.functional.binary_cross_entropy(
        recon_x, x.view(-1, 28*28), reduction='sum'
    )

    kl_loss = -0.5 * torch.sum(1 + logvar - mu.pow(2) - logvar.exp())

    return recon_loss + kl_loss


In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

model = VAE(latent_dim=20).to(device)
optimizer = optim.Adam(model.parameters(), lr=1e-3)

epochs = 200
train_losses = []

for epoch in range(epochs):
    model.train()
    total_loss = 0

    for x, _ in train_loader:
        x = x.to(device)
        optimizer.zero_grad()

        recon, mu, logvar = model(x)
        loss = vae_loss(recon, x, mu, logvar)

        loss.backward()
        optimizer.step()

        total_loss += loss.item()

    avg_loss = total_loss / len(train_loader.dataset)
    train_losses.append(avg_loss)
    print(f"Epoch {epoch+1}/{epochs}, Loss: {avg_loss:.4f}")


In [ ]:
model.eval()
with torch.no_grad():
    z = torch.randn(16, 20).to(device)
    samples = model.decoder(z).view(-1, 1, 28, 28).cpu()

plt.figure(figsize=(6,6))
for i in range(16):
    plt.subplot(4,4,i+1)
    plt.imshow(samples[i][0], cmap='gray')
    plt.axis('off')
plt.show()


In [ ]:
model_2d = VAE(latent_dim=2).to(device)
optimizer = optim.Adam(model_2d.parameters(), lr=1e-3)


In [ ]:
model_2d.eval()
latents = []
labels = []

with torch.no_grad():
    for x, y in test_loader:
        x = x.to(device)
        _, mu, _ = model_2d(x)
        latents.append(mu.cpu())
        labels.append(y)

latents = torch.cat(latents)
labels = torch.cat(labels)

plt.figure(figsize=(8,6))
plt.scatter(latents[:,0], latents[:,1], c=labels, cmap='tab10', s=5)
plt.colorbar()
plt.show()


In [ ]:
import os

output_dir = './saved_models'
os.makedirs(output_dir, exist_ok=True)

model_path = os.path.join(output_dir, 'vae_model.pth')
torch.save(model.state_dict(), model_path)

print(f"Model saved to: {model_path}")

In [ ]:
plt.figure(figsize=(10, 5))
plt.plot(train_losses)
plt.title('Training Loss over Epochs')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.grid(True)
plt.show()